# 🧠 TraderMind AI Training (Free Tier)

This notebook uses **Unsloth** to fine-tune Llama 3 (8B) on your custom `tradermind_finetune.jsonl` dataset.
It is optimized to run on a **free Google Colab T4 GPU**.

### Steps:
1. Upload `tradermind_finetune.jsonl` to the Files tab (left sidebar).
2. Run all cells.
3. Download the `GGUF` model file at the end.
4. Import into Ollama locally.

In [ ]:
%%capture
# 1. Install Unsloth and Dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# 2. Initialize Model
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # Auto detecion
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Optimized Llama 3
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)

In [ ]:
# 3. Load & Format Data
from datasets import load_dataset

dataset = load_dataset("json", data_files="tradermind_finetune.jsonl", split="train")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 4. Train Model
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100, # Quick run (Use 500+ for full convergence)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

In [ ]:
# 5. Export for Ollama
# This saves the model in GGUF format (q4_k_m) which is perfect for Ollama
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")